<a href="https://colab.research.google.com/github/kajalns/semeval2025-codegen-detection/blob/main/training_baseline_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install kaggle

import os, shutil
from google.colab import files

os.makedirs('/root/.kaggle', exist_ok=True)
print("Upload kaggle.json ")
uploaded = files.upload()

fname = next(iter(uploaded.keys()))
shutil.move(fname, '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 0o600)

!kaggle --version
!kaggle datasets list -s "semeval task 13" | head -n 10



Upload kaggle.json 


Saving kaggle.json to kaggle.json
Kaggle API 1.7.4.5
ref                           title                      size  lastUpdated                 downloadCount  voteCount  usabilityRating  
----------------------------  -------------------  ----------  --------------------------  -------------  ---------  ---------------  
daniilor/semeval-2026-task13  SemEval-2026-Task13  1106511457  2025-08-28 10:34:38.227000            278          6  0.47058824       


In [ ]:
SLUG = "daniilor/semeval-2026-task13"
TARGET = "/content/semeval_task13"
!mkdir -p "$TARGET"
!kaggle datasets download -d "$SLUG" -p "$TARGET"
!unzip -o "$TARGET"/*.zip -d "$TARGET"


Dataset URL: https://www.kaggle.com/datasets/daniilor/semeval-2026-task13
License(s): unknown
 98% 1.01G/1.03G [00:05<00:00, 154MB/s]
100% 1.03G/1.03G [00:05<00:00, 212MB/s]
Archive:  /content/semeval_task13/semeval-2026-task13.zip
  inflating: /content/semeval_task13/SemEval-2026-Task13/task_a/task_a_test_set_sample.parquet  
  inflating: /content/semeval_task13/SemEval-2026-Task13/task_a/task_a_training_set_1.parquet  
  inflating: /content/semeval_task13/SemEval-2026-Task13/task_a/task_a_validation_set.parquet  
  inflating: /content/semeval_task13/SemEval-2026-Task13/task_b/task_b_test_set_sample.parquet  
  inflating: /content/semeval_task13/SemEval-2026-Task13/task_b/task_b_training_set.parquet  
  inflating: /content/semeval_task13/SemEval-2026-Task13/task_b/task_b_validation_set.parquet  
  inflating: /content/semeval_task13/SemEval-2026-Task13/task_c/task_c_test_set_sample.parquet  
  inflating: /content/semeval_task13/SemEval-2026-Task13/task_c/task_c_training_set_1.parquet  

In [ ]:
#parquet and ML dependencies
!pip -q install pyarrow pandas scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
import os
import glob
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack, csr_matrix
import warnings
warnings.filterwarnings('ignore')

In [ ]:
BASE = Path("/content/semeval_task13/SemEval-2026-Task13/task_a")

train_path = BASE / "task_a_training_set_1.parquet"
val_path   = BASE / "task_a_validation_set.parquet"
test_path  = BASE / "task_a_test_set_sample.parquet"
print("dataset for task a")


dataset for task a


In [ ]:
# Load the datasets
train_df = pd.read_parquet(train_path)
val_df = pd.read_parquet(val_path)
test_df = pd.read_parquet(test_path)

print("Successfully loaded the datasets!")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")


Successfully loaded the datasets!
Training samples: 500000
Validation samples: 100000
Test samples: 1000


In [ ]:
# Exploration of data
print(f"\ncolumn names: {train_df.columns.tolist()}")
train_df.head()

# Verify labels are already numeric
print(f"\nLabel verification:")
print(f"Training labels: {train_df['label'].unique()} (dtype: {train_df['label'].dtype})")
print(f"Validation labels: {val_df['label'].unique()} (dtype: {val_df['label'].dtype})")

print(f"\n Training label distribution:")
print(train_df['label'].value_counts().sort_index())
print(f"\nValidation label distribution:")
print(val_df['label'].value_counts().sort_index())

print(f"\nLanguages in the dataset: {train_df['language'].unique()}")
print(f"Generators in the dataset: {train_df['generator'].unique()}")


column names: ['code', 'generator', 'label', 'language']

Label verification:
Training labels: [0 1] (dtype: int64)
Validation labels: [0 1] (dtype: int64)

 Training label distribution:
label
0    238475
1    261525
Name: count, dtype: int64

Validation label distribution:
label
0    47695
1    52305
Name: count, dtype: int64

Languages in the dataset: ['Python' 'Java' 'C++']
Generators in the dataset: ['human' 'Qwen/Qwen2.5-Coder-1.5B' 'Qwen/Qwen2.5-Coder-7B-Instruct'
 'microsoft/phi-2' 'microsoft/Phi-3-medium-4k-instruct'
 'codellama/CodeLlama-70b-Instruct-hf' 'microsoft/Phi-3.5-mini-instruct'
 'codellama/CodeLlama-34b-Instruct-hf' '01-ai/Yi-Coder-9B'
 'google/codegemma-7b' 'microsoft/Phi-3-mini-4k-instruct'
 'ibm-granite/granite-8b-code-base-4k' 'microsoft/Phi-3-small-8k-instruct'
 'meta-llama/Llama-3.2-1B' 'bigcode/starcoder'
 'meta-llama/Llama-3.1-8B-Instruct'
 'ibm-granite/granite-8b-code-instruct-4k' '01-ai/Yi-Coder-1.5B-Chat'
 '01-ai/Yi-Coder-9B-Chat' 'bigcode/starcoder2-15b'

In [ ]:
print("Subtask A")
print("Phase 1: Baseline model")

# Feature Extraction

def extract_features(train_df, val_df, text_col='code'):
    print("\nExtracting Features from the dataset")
    features = {}

    # 1.TF-IDF Features with different n-gram ranges
    print("1.TF-IDF Features")
    tfidf_configs = {
        'tfidf_1gram': TfidfVectorizer(max_features=3000, ngram_range=(1, 1), min_df=3),
        'tfidf_2gram': TfidfVectorizer(max_features=2000, ngram_range=(1, 2), min_df=3),
        'tfidf_3gram': TfidfVectorizer(max_features=1000, ngram_range=(1, 3), min_df=5),
    }

    for name, vectorizer in tfidf_configs.items():
        features[f'X_train_{name}'] = vectorizer.fit_transform(train_df[text_col])
        features[f'X_val_{name}'] = vectorizer.transform(val_df[text_col])
        features[f'vectorizer_{name}'] = vectorizer
        print(f"{name}: {features[f'X_train_{name}'].shape[1]} features")

     # 2.Count Vectorizer
    count_vectorizer = CountVectorizer(max_features=2000, ngram_range=(1, 2), min_df=2)
    features['X_train_count'] = count_vectorizer.fit_transform(train_df[text_col])
    features['X_val_count'] = count_vectorizer.transform(val_df[text_col])
    features['vectorizer_count'] = count_vectorizer
    print(f"Count Vectorizer: {features['X_train_count'].shape[1]} features")


    # 3.Code Statistics Features
    def code_statistics_features(code_text):
        lines = code_text.split('\n')
        words = code_text.split()

        return [
            # Basic metrics
            len(code_text), #total characters
            len(lines),     #number of lines
            len(words),     #number of words
            np.mean([len(line) for line in lines]) if lines else 0,  #avg line length

            # Code structure
            code_text.count('def '),   #function definitions
            code_text.count('class '), #class definitions
            code_text.count('import '),#import statements
            code_text.count('from '),  #from imports
            code_text.count('#'),      #comments
            code_text.count('print'),  #print statements

            # Syntax elements
            code_text.count('='),  #assignments
            code_text.count('('),  #open parentheses
            code_text.count(')'),  #colsed parentheses
            code_text.count('['),  #square brackets
            code_text.count(']'),   #square brackets
            code_text.count(':'),  #colons
            code_text.count(','),  #commas

            # Whitespace analysis
            code_text.count(' '),  #spaces
            code_text.count('\t'), #tabs
            code_text.count('\n'), #newlines

            # Complexity indicators
            code_text.count('if '),   #if statements
            code_text.count('for '),  #for loops
            code_text.count('while '), #while loops
            code_text.count('return '),#return statements

            # Lexical diversity
            len(set(words)) / len(words) if words else 0, #type-token ratio
        ]

    train_stats = np.array([code_statistics_features(code) for code in train_df[text_col]])
    val_stats = np.array([code_statistics_features(code) for code in val_df[text_col]])

    features['X_train_stats'] = train_stats
    features['X_val_stats'] = val_stats
    print(f"Code Statistics: {train_stats.shape[1]} features")

    #4.Combined Features (TF-IDF and Statistics)
    features['X_train_combined'] = hstack([
        features['X_train_tfidf_2gram'],
        csr_matrix(features['X_train_stats'])
    ])
    features['X_val_combined'] = hstack([
        features['X_val_tfidf_2gram'],
        csr_matrix(features['X_val_stats'])
    ])
    print(f"Combined: {features['X_train_combined'].shape[1]} features")

    #Labels
    features['y_train'] = train_df['label']
    features['y_val'] = val_df['label']

    print("Successfully completed the feature extraction")
    return features

#Extract features
features = extract_features(train_df, val_df, 'code')


Subtask A
Phase 1: Baseline model

Extracting Features from the dataset
1.TF-IDF Features
tfidf_1gram: 3000 features
tfidf_2gram: 2000 features
tfidf_3gram: 1000 features
Count Vectorizer: 2000 features
Code Statistics: 25 features
Combined: 2025 features
Successfully completed the feature extraction!


In [ ]:
#Baseline model1: Logistic regression
def logistic_regression_model(features):
    print("\nLOGISTIC REGRESSION")

    # Define model
    model = LogisticRegression(random_state=42, max_iter=1000, n_jobs=-1)
    # Feature sets
    feature_sets = {
        'TF-IDF_1gram': ('X_train_tfidf_1gram', 'X_val_tfidf_1gram'),
        'TF-IDF_2gram': ('X_train_tfidf_2gram', 'X_val_tfidf_2gram'),
        'TF-IDF_3gram': ('X_train_tfidf_3gram', 'X_val_tfidf_3gram'),
        'Count_vectorizer': ('X_train_count', 'X_val_count'),
        'Statistics_only': ('X_train_stats', 'X_val_stats'),
        'Combined': ('X_train_combined', 'X_val_combined')
    }
    results = {}

    for feature_name, (train_feat, val_feat) in feature_sets.items():
        X_train = features[train_feat]
        X_val = features[val_feat]
        y_train = features['y_train']
        y_val = features['y_val']

        try:
            # Train model
            model.fit(X_train, y_train)

            # Predictions
            val_preds = model.predict(X_val)

            # Metrics
            val_acc = accuracy_score(y_val, val_preds)
            val_f1 = f1_score(y_val, val_preds, average='binary')

            results[feature_name] = {
                'model': model,
                'val_accuracy': val_acc,
                'val_f1': val_f1,
                'predictions': val_preds
            }

        except Exception as e:
            print(f"Error: {str(e)[:30]}")
            continue

    print("Logistic regression training completed")
    return results

# Run Logistic Regression
lr_results = logistic_regression_model(features)
print("\nResult Summary:")
for feature, result in lr_results.items():
    print(f"  {feature:20} - Accuracy: {result['val_accuracy']:.4f}, F1: {result['val_f1']:.4f}")


LOGISTIC REGRESSION
Logistic regression training completed

Result Summary:
  TF-IDF_1gram         - Accuracy: 0.8604, F1: 0.8564
  TF-IDF_2gram         - Accuracy: 0.8689, F1: 0.8665
  TF-IDF_3gram         - Accuracy: 0.8629, F1: 0.8602
  Count_vectorizer     - Accuracy: 0.8654, F1: 0.8598
  Statistics_only      - Accuracy: 0.8942, F1: 0.8967
  Combined             - Accuracy: 0.8936, F1: 0.8954


In [ ]:
# Baseline model2:Linear SVM Model
def linear_svc_model(features):
    print("\nLINEAR SVC")

    # Define model
    model = LinearSVC(random_state=42, max_iter=1000)
    # Feature sets
    feature_sets = {
        'TF-IDF_1gram': ('X_train_tfidf_1gram', 'X_val_tfidf_1gram'),
        'TF-IDF_2gram': ('X_train_tfidf_2gram', 'X_val_tfidf_2gram'),
        'TF-IDF_3gram': ('X_train_tfidf_3gram', 'X_val_tfidf_3gram'),
        'Count_vectorizer': ('X_train_count', 'X_val_count'),
        'Statistics_only': ('X_train_stats', 'X_val_stats'),
        'Combined': ('X_train_combined', 'X_val_combined')
    }
    results = {}

    for feature_name, (train_feat, val_feat) in feature_sets.items():
        X_train = features[train_feat]
        X_val = features[val_feat]
        y_train = features['y_train']
        y_val = features['y_val']

        try:
            # Train model
            model.fit(X_train, y_train)

            # Predictions
            val_preds = model.predict(X_val)

            # Metrics
            val_acc = accuracy_score(y_val, val_preds)
            val_f1 = f1_score(y_val, val_preds, average='binary')

            results[feature_name] = {
                'model': model,
                'val_accuracy': val_acc,
                'val_f1': val_f1,
                'predictions': val_preds
            }

        except Exception as e:
            print(f"Error: {str(e)[:30]}...")
            continue

    print("Linear SVC training completed")
    return results

# Run Linear SVC
linear_svc_results = linear_svc_model(features)
print("\nResult Summary:")
for feature, result in linear_svc_results.items():
    print(f"  {feature:20} - Accuracy: {result['val_accuracy']:.4f}, F1: {result['val_f1']:.4f}")


LINEAR SVC
Linear SVC training completed

Result Summary:
  TF-IDF_1gram         - Accuracy: 0.8597, F1: 0.8539
  TF-IDF_2gram         - Accuracy: 0.8705, F1: 0.8669
  TF-IDF_3gram         - Accuracy: 0.8635, F1: 0.8590
  Count_vectorizer     - Accuracy: 0.8589, F1: 0.8507
  Statistics_only      - Accuracy: 0.8547, F1: 0.8664
  Combined             - Accuracy: 0.8857, F1: 0.8859


In [ ]:
# Baseline model 3: Random Forest
def random_forest_model(features):
    print("\nRandom Forest Model")

    # Define model
    model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

    # Feature sets
    feature_sets = {
        'TF-IDF_1gram': ('X_train_tfidf_1gram', 'X_val_tfidf_1gram'),
        'TF-IDF_2gram': ('X_train_tfidf_2gram', 'X_val_tfidf_2gram'),
        'TF-IDF_3gram': ('X_train_tfidf_3gram', 'X_val_tfidf_3gram'),
        'Count_vectorizer': ('X_train_count', 'X_val_count'),
        'Statistics_only': ('X_train_stats', 'X_val_stats'),
        'Combined': ('X_train_combined', 'X_val_combined')
    }

    results = {}

    for feature_name, (train_feat, val_feat) in feature_sets.items():
        X_train = features[train_feat]
        X_val = features[val_feat]
        y_train = features['y_train']
        y_val = features['y_val']

        try:
            # Train model
            model.fit(X_train, y_train)

            # Predictions
            val_preds = model.predict(X_val)

            # Metrics
            val_acc = accuracy_score(y_val, val_preds)
            val_f1 = f1_score(y_val, val_preds, average='binary')

            results[feature_name] = {
                'model': model,
                'val_accuracy': val_acc,
                'val_f1': val_f1,
                'predictions': val_preds
            }

        except Exception as e:
            print(f"Error: {str(e)[:30]}...")
            continue

    print("Random Forest training completed")
    return results

# Run Random Forest
rf_results = random_forest_model(features)
print("\nResults Summary:")
for feature, result in rf_results.items():
    print(f"  {feature:20} - Accuracy: {result['val_accuracy']:.4f}, F1: {result['val_f1']:.4f}")


Random Forest Model


KeyboardInterrupt: 

In [ ]:
# Baseline model4:Decsion Tree Model
def decision_tree_model(features):
    print("\nDECISION TREE MODEL")

    # Define model
    model = DecisionTreeClassifier(random_state=42)
    # Feature sets
    feature_sets = {
        'TF-IDF_1gram': ('X_train_tfidf_1gram', 'X_val_tfidf_1gram'),
        'TF-IDF_2gram': ('X_train_tfidf_2gram', 'X_val_tfidf_2gram'),
        'TF-IDF_3gram': ('X_train_tfidf_3gram', 'X_val_tfidf_3gram'),
        'Count_vectorizer': ('X_train_count', 'X_val_count'),
        'Statistics_only': ('X_train_stats', 'X_val_stats'),
        'Combined': ('X_train_combined', 'X_val_combined')
    }
    results = {}

    for feature_name, (train_feat, val_feat) in feature_sets.items():

        X_train = features[train_feat]
        X_val = features[val_feat]
        y_train = features['y_train']
        y_val = features['y_val']

        try:
            # Train model
            model.fit(X_train, y_train)

            # Predictions
            val_preds = model.predict(X_val)

            # Metrics
            val_acc = accuracy_score(y_val, val_preds)
            val_f1 = f1_score(y_val, val_preds, average='binary')

            results[feature_name] = {
                'model': model,
                'val_accuracy': val_acc,
                'val_f1': val_f1,
                'predictions': val_preds
            }

        except Exception as e:
            print(f"Error: {str(e)[:30]}...")
            continue

    print("Decision Tree training completed")
    return results

# Run Decision Tree
dt_results = decision_tree_model(features)
print("\nResults Summary:")
for feature, result in dt_results.items():
    print(f"  {feature:20} - Accuracy: {result['val_accuracy']:.4f}, F1: {result['val_f1']:.4f}")


DECISION TREE MODEL
Decision Tree training completed

Results Summary:
  TF-IDF_1gram         - Accuracy: 0.8763, F1: 0.8806
  TF-IDF_2gram         - Accuracy: 0.8802, F1: 0.8849
  TF-IDF_3gram         - Accuracy: 0.8763, F1: 0.8811
  Count_vectorizer     - Accuracy: 0.8943, F1: 0.8982
  Statistics_only      - Accuracy: 0.9538, F1: 0.9559
  Combined             - Accuracy: 0.9706, F1: 0.9719


In [ ]:
# Baseline model5: Gradient Boosting Model
def gradient_boosting_model(features):
    print("\nGRADIENT BOOSTING")

    # Define model
    model = GradientBoostingClassifier(n_estimators=100, random_state=42)
    # Feature sets
    feature_sets = {
        'TF-IDF_1gram': ('X_train_tfidf_1gram', 'X_val_tfidf_1gram'),
        'TF-IDF_2gram': ('X_train_tfidf_2gram', 'X_val_tfidf_2gram'),
        'TF-IDF_3gram': ('X_train_tfidf_3gram', 'X_val_tfidf_3gram'),
        'Count_vectorizer': ('X_train_count', 'X_val_count'),
        'Statistics_only': ('X_train_stats', 'X_val_stats'),
        'Combined': ('X_train_combined', 'X_val_combined')
    }
    results = {}

    for feature_name, (train_feat, val_feat) in feature_sets.items():
        X_train = features[train_feat]
        X_val = features[val_feat]
        y_train = features['y_train']
        y_val = features['y_val']

        try:
            # Train model
            model.fit(X_train, y_train)

            # Predictions
            val_preds = model.predict(X_val)

            # Metrics
            val_acc = accuracy_score(y_val, val_preds)
            val_f1 = f1_score(y_val, val_preds, average='binary')

            results[feature_name] = {
                'model': model,
                'val_accuracy': val_acc,
                'val_f1': val_f1,
                'predictions': val_preds
            }

        except Exception as e:
            print(f"Error: {str(e)[:30]}...")
            continue

    print("Gradient Boosting training completed")
    return results

# Run Gradient Boosting
gb_results = gradient_boosting_model(features)
print("\nResults Summary:")
for feature, result in gb_results.items():
    print(f"  {feature:20} - Accuracy: {result['val_accuracy']:.4f}, F1: {result['val_f1']:.4f}")


GRADIENT BOOSTING


KeyboardInterrupt: 

In [ ]:
# Baseline model6: AdaBoost Model
def adaboost_model(features):
    print("\nADABOOST Model")

    # Define model
    model = AdaBoostClassifier(n_estimators=100, random_state=42)
    # Feature sets
    feature_sets = {
        'TF-IDF_1gram': ('X_train_tfidf_1gram', 'X_val_tfidf_1gram'),
        'TF-IDF_2gram': ('X_train_tfidf_2gram', 'X_val_tfidf_2gram'),
        'TF-IDF_3gram': ('X_train_tfidf_3gram', 'X_val_tfidf_3gram'),
        'Count_vectorizer': ('X_train_count', 'X_val_count'),
        'Statistics_only': ('X_train_stats', 'X_val_stats'),
        'Combined': ('X_train_combined', 'X_val_combined')
    }

    results = {}

    for feature_name, (train_feat, val_feat) in feature_sets.items():
        X_train = features[train_feat]
        X_val = features[val_feat]
        y_train = features['y_train']
        y_val = features['y_val']

        try:
            # Train model
            model.fit(X_train, y_train)

            # Predictions
            val_preds = model.predict(X_val)

            # Metrics
            val_acc = accuracy_score(y_val, val_preds)
            val_f1 = f1_score(y_val, val_preds, average='binary')

            results[feature_name] = {
                'model': model,
                'val_accuracy': val_acc,
                'val_f1': val_f1,
                'predictions': val_preds
            }

        except Exception as e:
            print(f"Error: {str(e)[:30]}...")
            continue

    print("AdaBoost training completed")
    return results

# Run AdaBoost
ada_results = adaboost_model(features)
print("\nResults Summary:")
for feature, result in ada_results.items():
    print(f"  {feature:20} - Accuracy: {result['val_accuracy']:.4f}, F1: {result['val_f1']:.4f}")


ADABOOST Model


KeyboardInterrupt: 

In [ ]:
# Baseline Model 8: K-NEIGHBORS ONLY
def kneighbors_model(features):
    print("\nKNN Model")

    # Define model
    model = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
    # Feature sets
    feature_sets = {
        'TF-IDF_1gram': ('X_train_tfidf_1gram', 'X_val_tfidf_1gram'),
        'TF-IDF_2gram': ('X_train_tfidf_2gram', 'X_val_tfidf_2gram'),
        'TF-IDF_3gram': ('X_train_tfidf_3gram', 'X_val_tfidf_3gram'),
        'Count_vectorizer': ('X_train_count', 'X_val_count'),
        'Statistics_only': ('X_train_stats', 'X_val_stats'),
        'Combined': ('X_train_combined', 'X_val_combined')
    }
    results = {}

    for feature_name, (train_feat, val_feat) in feature_sets.items():

        X_train = features[train_feat]
        X_val = features[val_feat]
        y_train = features['y_train']
        y_val = features['y_val']

        try:
            # Train model
            model.fit(X_train, y_train)

            # Predictions
            val_preds = model.predict(X_val)

            # Metrics
            val_acc = accuracy_score(y_val, val_preds)
            val_f1 = f1_score(y_val, val_preds, average='binary')

            results[feature_name] = {
                'model': model,
                'val_accuracy': val_acc,
                'val_f1': val_f1,
                'predictions': val_preds
            }


        except Exception as e:
            print(f"Error: {str(e)[:30]}...")
            continue

    print("K-Neighbors training completed")
    return results

# Run K-Neighbors
knn_results = kneighbors_model(features)
print("\nResults Summary:")
for feature, result in knn_results.items():
    print(f"  {feature:20} - Accuracy: {result['val_accuracy']:.4f}, F1: {result['val_f1']:.4f}")

In [ ]:
# Baseline Model 10: XGBOOST
def xgboost_model(features):
    print("XGBOOST MOdel")

    # Define model
    try:
        from xgboost import XGBClassifier
        model = XGBClassifier(
            n_estimators=100,
            random_state=42,
            n_jobs=-1,
            eval_metric='logloss',
            use_label_encoder=False
        )
    except ImportError:
        print("installation error")
        return {}

    # Feature sets
    feature_sets = {
        'TF-IDF_1gram': ('X_train_tfidf_1gram', 'X_val_tfidf_1gram'),
        'TF-IDF_2gram': ('X_train_tfidf_2gram', 'X_val_tfidf_2gram'),
        'TF-IDF_3gram': ('X_train_tfidf_3gram', 'X_val_tfidf_3gram'),
        'Count_vectorizer': ('X_train_count', 'X_val_count'),
        'Statistics_only': ('X_train_stats', 'X_val_stats'),
        'Combined': ('X_train_combined', 'X_val_combined')
    }

    results = {}

    for feature_name, (train_feat, val_feat) in feature_sets.items():

        X_train = features[train_feat]
        X_val = features[val_feat]
        y_train = features['y_train']
        y_val = features['y_val']

        try:
            # Train model
            model.fit(X_train, y_train)

            # Predictions
            val_preds = model.predict(X_val)

            # Metrics
            val_acc = accuracy_score(y_val, val_preds)
            val_f1 = f1_score(y_val, val_preds, average='binary')

            results[feature_name] = {
                'model': model,
                'val_accuracy': val_acc,
                'val_f1': val_f1,
                'predictions': val_preds
            }


        except Exception as e:
            print(f"Error: {str(e)[:30]}...")
            continue

    print("XGBoost training completed")
    return results

# Run XGBoost
xgb_results = xgboost_model(features)
print("\nResults Summary:")
for feature, result in xgb_results.items():
    print(f"  {feature:20} - Accuracy: {result['val_accuracy']:.4f}, F1: {result['val_f1']:.4f}")

XGBOOST MOdel
XGBoost training completed

Results Summary:
  TF-IDF_1gram         - Accuracy: 0.9044, F1: 0.9026
  TF-IDF_2gram         - Accuracy: 0.9091, F1: 0.9082
  TF-IDF_3gram         - Accuracy: 0.9096, F1: 0.9089
  Count_vectorizer     - Accuracy: 0.9193, F1: 0.9190
  Statistics_only      - Accuracy: 0.9711, F1: 0.9722
  Combined             - Accuracy: 0.9816, F1: 0.9824


In [ ]:
# Baseline Model 7: Naive Bayes Model
def naive_bayes_model(features):
    print("\nNAIVE BAYES MODEL")

    # Define model
    model = MultinomialNB()
    # Feature sets
    feature_sets = {
        'TF-IDF_1gram': ('X_train_tfidf_1gram', 'X_val_tfidf_1gram'),
        'TF-IDF_2gram': ('X_train_tfidf_2gram', 'X_val_tfidf_2gram'),
        'TF-IDF_3gram': ('X_train_tfidf_3gram', 'X_val_tfidf_3gram'),
        'Count_vectorizer': ('X_train_count', 'X_val_count'),
        'Statistics_only': ('X_train_stats', 'X_val_stats'),
        'Combined': ('X_train_combined', 'X_val_combined')
    }
    results = {}

    for feature_name, (train_feat, val_feat) in feature_sets.items():

        X_train = features[train_feat]
        X_val = features[val_feat]
        y_train = features['y_train']
        y_val = features['y_val']

        try:
            # Train model
            model.fit(X_train, y_train)

            # Predictions
            val_preds = model.predict(X_val)

            # Metrics
            val_acc = accuracy_score(y_val, val_preds)
            val_f1 = f1_score(y_val, val_preds, average='binary')

            results[feature_name] = {
                'model': model,
                'val_accuracy': val_acc,
                'val_f1': val_f1,
                'predictions': val_preds
            }


        except Exception as e:
            print(f"Error: {str(e)[:30]}...")
            continue

    print("Naive Bayes training completed")
    return results

# Run Naive Bayes
nb_results = naive_bayes_model(features)
print("\nResults Summary:")
for feature, result in nb_results.items():
    print(f"  {feature:20} - Accuracy: {result['val_accuracy']:.4f}, F1: {result['val_f1']:.4f}")


NAIVE BAYES MODEL
Naive Bayes training completed

Results Summary:
  TF-IDF_1gram         - Accuracy: 0.7983, F1: 0.7917
  TF-IDF_2gram         - Accuracy: 0.8092, F1: 0.8005
  TF-IDF_3gram         - Accuracy: 0.8018, F1: 0.7956
  Count_vectorizer     - Accuracy: 0.7937, F1: 0.7705
  Statistics_only      - Accuracy: 0.8748, F1: 0.8771
  Combined             - Accuracy: 0.8753, F1: 0.8776


In [ ]:
# Baseline Model 9: SVC with RBF Kernel
def svc_rbf_model(features):
    print("\nSVC WITH RBF KERNEL")

    # Define model
    model = SVC(kernel='rbf', random_state=42, probability=True)

    # Feature sets
    feature_sets = {
        'TF-IDF_1gram': ('X_train_tfidf_1gram', 'X_val_tfidf_1gram'),
        'TF-IDF_2gram': ('X_train_tfidf_2gram', 'X_val_tfidf_2gram'),
        'TF-IDF_3gram': ('X_train_tfidf_3gram', 'X_val_tfidf_3gram'),
        'Count_vectorizer': ('X_train_count', 'X_val_count'),
        'Statistics_only': ('X_train_stats', 'X_val_stats'),
        'Combined': ('X_train_combined', 'X_val_combined')
    }

    results = {}

    for feature_name, (train_feat, val_feat) in feature_sets.items():

        X_train = features[train_feat]
        X_val = features[val_feat]
        y_train = features['y_train']
        y_val = features['y_val']

        try:
            # Train model
            model.fit(X_train, y_train)

            # Predictions
            val_preds = model.predict(X_val)

            # Metrics
            val_acc = accuracy_score(y_val, val_preds)
            val_f1 = f1_score(y_val, val_preds, average='binary')

            results[feature_name] = {
                'model': model,
                'val_accuracy': val_acc,
                'val_f1': val_f1,
                'predictions': val_preds
            }

        except Exception as e:
            print(f"Error: {str(e)[:30]}...")
            continue

    print("SVC RBF training completed")
    return results

# Run SVC RBF
svc_rbf_results = svc_rbf_model(features)
print("\nResults Summary:")
for feature, result in svc_rbf_results.items():
    print(f"  {feature:20} - Accuracy: {result['val_accuracy']:.4f}, F1: {result['val_f1']:.4f}")


SVC WITH RBF KERNEL
